In [1]:
import pandas as pd
from pandas import DataFrame
from functools import partial

In [2]:
%pwd
%cd "~/Documents/GitHub/smart-tab-grouping"


/Users/Rrando/Documents/GitHub/smart-tab-grouping


/Users/Rrando/Documents/GitHub/smart-tab-grouping/.venv/lib/python3.11/site-packages/IPython/core/magics/osm.py:393: UserWarning: This is now an optional IPython functionality, using bookmarks requires you to install the `pickleshare` library.
  bkms = self.shell.db.get('bookmarks', {})
/Users/Rrando/Documents/GitHub/smart-tab-grouping/.venv/lib/python3.11/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [3]:
from rouge_score import rouge_scorer

In [4]:
df = pd.read_csv("data/individual_tests/private/all_users2.csv")

In [5]:
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

In [6]:
def compute_rouge_scores(row, pred_key=None):
    scores = scorer.score(row['label'], row[pred_key])
    return {
        'rouge1': scores['rouge1'].fmeasure,
        'rouge2': scores['rouge2'].fmeasure,
        'rougeL': scores['rougeL'].fmeasure,
        'pred_len': len(row[pred_key]),
        'label_len': len(row['label'])
    }


In [7]:
def print_rouge_scores(input_df: DataFrame, compare_column: str):
    rouge_scores_df = input_df.apply(partial(compute_rouge_scores, pred_key=compare_column) , axis=1, result_type='expand')
    average_scores = rouge_scores_df.mean().to_dict()
    print(average_scores)


In [8]:
import sys
sys.path.append("/Users/Rrando/Documents/GitHub/smart-tab-grouping/src")
from util.tab_titles import T5TopicGenerator

In [9]:
topic_gen = T5TopicGenerator()

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


/Users/Rrando/Documents/GitHub/smart-tab-grouping


In [10]:
def compute_topic_keywords(row, legacy=True):
    return topic_gen.get_topic_with_keywords({"documents": row["three_titles"].split('\n'), "keywords": row["keywords"].split(',')}, legacy=legacy)

In [11]:
def compute_topic(row):
    return topic_gen.get_topic({"documents": row["three_titles"].split('\n')})

In [12]:
# Look at OpenAI can do for generating topics from a set of tabs

df["recomputed_titles_keywords"] = df.apply(lambda row: compute_topic_keywords(row), axis=1)
df["recomputed_title_no_keywords"] = df.apply(lambda row: compute_topic(row), axis=1)
llm_topic_gen_no_keywords = OpenAITopicGenerator(support_keywords=False)
df["openai_keywords"] = df.apply(lambda row: llm_topic_gen_keywords.get_topic({"documents": row.three_titles.split("\n"), "keywords": row.keywords.split(",")}), axis=1)
print_rouge_scores(df, "openai_keywords")

NameError: name 'OpenAITopicGenerator' is not defined

Compare with fine tuned model

In [13]:
topic_gen = T5TopicGenerator(model_name="./models/gentle-pyramid-114")
df["recomputed_title_keywords_pyramid"] = df.apply(lambda row: compute_topic_keywords(row, legacy=False), axis=1)
print_rouge_scores(df, "recomputed_title_keywords_pyramid")

/Users/Rrando/Documents/GitHub/smart-tab-grouping
{'rouge1': 0.31388888888888883, 'rouge2': 0.020833333333333332, 'rougeL': 0.31388888888888883, 'pred_len': 7.666666666666667, 'label_len': 11.375}


In [14]:
topic_gen = T5TopicGenerator(model_name="./models/cool-yogurt-98")
df["recomputed_title_keywords_yogurt"] = df.apply(lambda row: compute_topic_keywords(row, legacy=False), axis=1)
print_rouge_scores(df, "recomputed_title_keywords_yogurt")

/Users/Rrando/Documents/GitHub/smart-tab-grouping
{'rouge1': 0.32837301587301587, 'rouge2': 0.07222222222222223, 'rougeL': 0.32837301587301587, 'pred_len': 16.333333333333332, 'label_len': 11.375}


In [19]:
topic_gen = T5TopicGenerator(model_name="./models/swift-rain-107")
df["recomputed_title_keywords_rain"] = df.apply(lambda row: compute_topic_keywords(row, legacy=False), axis=1)
print_rouge_scores(df, "recomputed_title_keywords_rain")

/Users/Rrando/Documents/GitHub/smart-tab-grouping
{'rouge1': 0.2859126984126984, 'rouge2': 0.0375, 'rougeL': 0.2859126984126984, 'pred_len': 13.645833333333334, 'label_len': 11.375}


Look at relative length

In [20]:
df["recomputed_title_keywords_rain"].str.len().mean()

13.645833333333334

In [18]:
df["recomputed_title_keywords_pyramid"].str.len().mean()

7.666666666666667